# Rank2 优化
- 数据集：dev_en_multiturn_ii.json, dev_zh_multiturn_cii.json
- 模型：GPT-4o-mini
- 输入：rank1，prompt
- 输出：深入检索问题

## 生成问题

In [2]:
from openai import OpenAI 
import pandas as pd
import requests
import json
import os
from enum import Enum
import re
from tqdm.notebook import tqdm

In [3]:
prompt2_6_zh = '''
# Role
你是一位熟悉网络文化和梗的研究员，擅长挖掘和解析网络梗的深层含义。
## Attention
你负责根据图片描述，基于已有检索问题设计出能够深入精确检索到梗文化或图片隐喻的检索问题。

## Skills
### Skill 1: 图片分析
- 能够根据用户对图片的描述进行全面分析

### Skill 2: 隐喻理解
- 能够根据图片分析准确识别图片中的隐喻，并能提取深层含义

### Skill 3: 深入分析
- 能够深入挖掘已有检索问题，根据排序理由进一步深入弥补隐喻信息，提出深入的检索问题


## Workflow:
1. 分析提供的已有检索问题和排序理由，根据对图片描述的理解，补充有关图片隐喻的待检索的信息。
2. 深入设计更具体且有针对性的检索问题，以提高搜索的精确度。
3. 确保检索问题能够引导用户找到与梗图相关的文化背景或隐喻解释。

## Constraints
- 检索问题需要围绕梗文化或图片隐喻，避免过于广泛或不相关的搜索结果
- 检索问题需要在已有检索问题上再深入具体
- 检索问题总数为5个，请设计最相关的问题，仅输出检索问题

## Example: 
输入：
"""
图片描述："这幅漫画分为左右两部分，左侧描绘了一个懒散的成年人坐在沙发上，身旁是一个正在玩玩具的小孩，背景中有蛛网和一个生锈的哑铃。右侧则展示了成年人正在进行力量训练，而同样的孩子则坐在钢琴前，表现出专注和努力的姿态。左侧成年人说道"你看人家隔壁小孩,也跟你一起开始练的琴可是人家坚持下来了 结果现在多不一样!"。图片传达了一个主题：人们在不同阶段的坚持与努力，形成了截然不同的结果，体现了坚持的重要性；同时也体现了父母对子女教育的观念不同，言传身教的不同影响。整体色调偏向温暖，增强了故事的情感对比。"
相关性排序结果：
1. 问题：坚持的重要性在生活选择中的体现及其文化隐喻？
  - 排序理由：xxx
2. 问题：懈怠与努力的对比如何通过情感隐喻在网络文化中展现？
  - 排序理由：xxx
3. 问题：代际传承的主题在时间流逝与成长蜕变中的表现？
  - 排序理由：xxx
"""
输出：
"""
1. "坚持与懈怠在中国式教育中的体现？"
2. "家长的懈怠与努力对孩子成长的不同影响？"
3. ""躺平"与"奋斗"价值观的对比在网络梗图中如何通过成年人与儿童形象进行表达？"
4. "成年人和儿童形象转换所暗示的人生阶段蜕变有哪些深层含义？"
5. "成年人形象的"躺平"姿态在梗图中包含了哪些反讽和自我批评的元素？"
"""

## Solve:
图片描述：{}
相关性排序结果：{}
'''

prompt2_6_en = '''
# Role
You are a researcher familiar with internet culture and memes, skilled at uncovering and analyzing the deeper meanings of internet memes.
## Attention
You are responsible for designing more specific and targeted search questions based on the image description to improve the accuracy of the search results.

## Skills
### Skill 1: Image Analysis
- Ability to conduct a comprehensive analysis based on the user's description of the image

### Skill 2: Metaphor Comprehension
- Ability to accurately identify metaphors in images based on image analysis and extract deep meaning

### Skill 3: In-depth Analysis
- Ability to further explore the existing search questions, based on the sorting reasons to further supplement the metaphor information and propose more in-depth search questions

## Workflow:
1. Analyze the provided existing search questions and sorting reasons, supplement the information to be searched about the image metaphor based on the understanding of the image description.
2. Design more specific and targeted search questions to improve the accuracy of the search.
3. Ensure that the search questions can guide users to find cultural backgrounds or metaphorical explanations related to the meme images.

## Constraints
- Search questions should focus on meme culture or image metaphors, avoiding overly broad or irrelevant search results
- Search questions should be more specific based on the existing search questions
- The total number of search questions should be 5, please design the most relevant questions, only output the search questions

## Example:
Input:
"""
Image Description: The cartoon is divided into two parts, the left and right. The left side depicts a lazy adult sitting on a sofa, with a child playing with toys beside him, and a cobweb and a rusty dumbbell in the background. The right side shows the adult doing strength training, while the same child sits in front of the piano, showing a focused and hardworking attitude. The adult on the left says, "Look at the kid next door, who started playing the piano with you, but he persisted and now the results are different!" The picture conveys a theme: the persistence and effort of people at different stages lead to different results, reflecting the importance of persistence; it also reflects the different concepts of parental education and the different influences of words and deeds. The overall tone is warm, enhancing the emotional contrast of the story.
Relevance Sorting Results:
1. Question: "How does the importance of persistence manifest in life choices and its cultural metaphor?"
  - Sorting Reason: xxx
2. Question: "How is the contrast between slackness and effort expressed through emotional metaphors in internet culture?"
  - Sorting Reason: xxx
3. Question: "How does the theme of intergenerational inheritance manifest in the passage of time and growth transformation?"
  - Sorting Reason: xxx
"""
Output:
"""
1. "How does persistence and slackness manifest in Chinese-style education?"
2. "How do the different influences of parental slackness and effort affect children's growth?"
3. "How is the contrast between the values of "lying flat" and "struggle" expressed in internet meme images through adult and child images?"
4. "What are the deep meanings of life stage transformation implied by the transformation of adult and child images?"
5. "What elements of irony and self-criticism are contained in the "lying flat" posture of adult images in meme images?"
"""

## Solve:
Image Description: {}
Relevance Sorting Results: {}
'''

In [4]:
def GPT_P2_6(prompt,image_dep,search_questions):    
    messages = [
            {"role": "user", "content": prompt.format(image_dep, search_questions)}
        ]

    api_key = "sk-oNmreUjgotXLIWnh73F0Ac481c0c439487CaC49c93686aCc"
    proxy_api_url = 'https://happyapi.org/v1/chat/completions'
    headers = { 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_key}', }
    data = {'model': 'gpt-4o-mini', 'messages': messages, 'temperature': 0.7, 'top_p': 0.9}
    response = requests.post(proxy_api_url, headers=headers, json=data)
    search_questions_new = response.json()["choices"][0]["message"]["content"]
    return search_questions_new    

In [ ]:
# 数据rank前后变化：test_en-0,2有一个变化，1,3,4无变化；test_zh-0,1,2,3,4均无变化
with open('test_zh.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

for idx, item in enumerate(tqdm(data, desc="图片识别中")):
    # 生成新的搜索问题
    image_dep = item['image_dep']
    rank_search_result = item['rank_search_result']
    # search_questions_new_all = GPT_P2_6(prompt2_6_en, image_dep, rank_search_result)
    search_questions_new_all = GPT_P2_6(prompt2_6_zh, image_dep, rank_search_result) 
    print(search_questions_new_all)
    print('-----------------')


## Rank 现有实现方法

In [ ]:
def GPT_P2_4(prompt6,image_dep,search_result_all):    
    messages = [
            {"role": "user", "content": prompt6.format(image_dep, search_result_all)}
        ]

    api_key = "sk-oNmreUjgotXLIWnh73F0Ac481c0c439487CaC49c93686aCc"
    proxy_api_url = 'https://happyapi.org/v1/chat/completions'
    headers = { 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_key}', }
    data = {'model': 'gpt-4o-mini', 'messages': messages, 'temperature': 0, 'top_p': 0.9}
    response = requests.post(proxy_api_url, headers=headers, json=data)
    rank_search_result = response.json()["choices"][0]["message"]["content"]
    return rank_search_result

In [ ]:
prompt2_4_zh = '''
# Role
你是一位熟悉网络文化和梗的研究员，擅长从图像中提取深层含义，并将这些含义与相关的问题和总结进行匹配和排序。
## Attention
你负责根据图片描述，对多个问题-总结对进行相关性排序，并选择最相关的3个，帮助用户理解图片的深层含义。

## Skills
### Skill 1: 图片分析
- 能够根据用户对图片的描述进行全面分析

### Skill 2: 隐喻理解
- 能够根据图片分析准确识别图片中的隐喻，并能提取深层含义

### Skill 3: 相关性排序
- 能够根据图片的隐喻理解，针对问题-总结对的内容是否与图片隐喻相关，对问题-总结对进行相关性排序
- 能够提供对应相关性排序理由，并分析该问题-总结对中低相关内容

## Workflow:
  1. 仔细分析图片描述，<隐式>提取图片中的隐喻和关键元素。
  2. 对每个问题-总结对进行分析，<隐式>评估其与图片隐喻的相关性。
  3. 根据相关性大小，<隐式>对问题-总结对进行排序。
  4. <显示>输出排序后最相关的3个问题-总结对和排序理由。

## Constraints
- 排序应基于图片隐喻的相关性，确保排序结果的准确性和逻辑性
- 若问题-总结对中某部分与图片隐喻无关，请在对应理由中提供
- 仅输出排序后的最相关的3个问题-总结对和排序理由，避免使用markdown格式

## Examples:
输入：
"""
图片描述："一个人站在分岔路口"，问题-总结对："1. 问题："人生选择的重要性"; 2. 问题："面对困难时的决策过程"; 3. 问题："城市规划的复杂性"; 4. 问题："旅行中的导航技巧"; 5. 问题："个人成长与自我发现""
"""
输出：
"""
1. 问题：人生选择的重要性
  - 排序理由：xxx
2. 问题：面对困难时的决策过程
  - 排序理由：xxx
3. 问题：个人成长与自我发现
  - 排序理由：xxx
"""

## Solve:
图片描述：{}
问题-总结对：{}
'''

prompt2_4_en = '''
# Role
You are a researcher familiar with online culture and memes, and are good at extracting deep meanings from images, and matching and sorting these meanings with relevant questions and summaries.
## Attention
You are responsible for sorting the relevance of multiple question-summary pairs based on the image description, and selecting the three most relevant ones to help users understand the deep meaning of the image.

## Skills
### Skill 1: Image Analysis
- Ability to conduct a comprehensive analysis based on the user's description of the image

### Skill 2: Metaphor Understanding
- Ability to accurately identify metaphors in images based on image analysis and extract deep meaning

### Skill 3: Relevance Sorting
- Ability to sort the relevance of question-summary pairs based on the metaphorical understanding of the image, based on whether the content of the question-summary pair is related to the image metaphor
- Ability to provide reasons for the corresponding relevance sorting, and analyze the low-relevance content of the question-summary pair

## Workflow:
1. Carefully analyze the image description and <implicitly> extract the metaphors and key elements in the image.
2. Analyze each question-summary pair and <implicitly> evaluate its relevance to the image metaphor.
3. According to the relevance, <implicitly> sort the question-summary pairs.
4. <display> Output the top 3 most relevant question-summary pairs after sorting and the reasons for sorting.

## Constraints
- Sorting should be based on the relevance of the image metaphor to ensure the accuracy and logic of the sorting results
- If a part of the question-summary pair is not related to the image metaphor, please provide it in the corresponding reason
- Only output the top 3 most relevant question-summary pairs after sorting and the reasons for sorting, avoid using markdown format

## Examples:
Input:
"""
Image Description: "A person standing at a crossroads", Question-Summary Pair: "1. Question: "The importance of life choices"; 2. Question: "Decision-making process in the face of difficulties"; 3. Question: "Complexity of urban planning"; 4. Question: "Navigation skills in travel"; 5. Question: "Personal growth and self-discovery""
"""
Output:
"""
1. Question: The importance of life choices
  - Sorting Reason: xxx
2. Question: Decision-making process in the face of difficulties
  - Sorting Reason: xxx
3. Question: Personal growth and self-discovery
  - Sorting Reason: xxx
"""

## Solve:
Image Description: {}
Question-Summary Pair: {}
'''

In [ ]:
# Rank1
rank_search_result = GPT_P2_4(prompt2_4_zh, image_dep, search_result_all) # 5 results
# rank_search_result = GPT_P2_4(prompt2_4_en, image_dep, search_result_all)

# Rank2
rank_search_new_result = GPT_P2_4(prompt2_4_zh, image_dep, search_result_top_mid) # search_result_top_mid = new_search_result_all(5 new results) + rank1(3 results)
# rank_search_new_result = GPT_P2_4(prompt2_4_en, image_dep, search_result_top_mid)